In [0]:
%sql
create database if not exists dim;

In [0]:
%sql
create table if not exists dim.Customer(
CustomerID  varchar(20),  
CustomerName varchar(20) , 
Segment varchar(20), 
Country varchar(20) , 
City varchar(20), 
State varchar(20), 
PostalCode BIGINT , 
Region varchar(20),
createdDate timestamp,
updatedDate timestamp
)

In [0]:
%sql
-- merge into dim.Customer as tgt
-- using 
-- (Select distinct Customer_ID, Customer_Name, Segment, Country, City, State, Postal_Code,Region from cs_raw.invoice)
-- as src
-- on tgt.CustomerID = src.Customer_ID
-- when matched then 
--   update 
--      set 
--      tgt.CustomerName = src.Customer_Name, 
--      tgt.Segment = src.Segment, 
--      tgt.Country = src.Country, 
--      tgt.City = src.City, 
--      tgt.State = src.State, 
--      tgt.PostalCode=src.Postal_Code,
--      tgt.Region = src.Region,
--      tgt.updatedDate=current_timestamp()
-- when not matched then
--  INSERT (CustomerID, CustomerName, Segment, Country, City, State, PostalCode, Region, createdDate, updatedDate)
--   VALUES (src.Customer_ID, src.Customer_Name, src.Segment, src.Country, src.City, src.State, src.Postal_Code, src.Region, current_timestamp(), current_timestamp());



In [0]:
%sql
Select current_date

current_date()
2025-01-31


In [0]:
from pyspark.sql.functions import current_timestamp

# Load the source DataFrame from the raw invoice table
src_df = spark.sql("""
    SELECT DISTINCT 
        Customer_ID, Customer_Name, Segment, Country, City, State, Postal_Code, Region 
    FROM cs_raw.invoice
""")

# Define the target table path (if using Delta Lake)
target_table = "dim.Customer"

# Perform the merge operation
from delta.tables import DeltaTable

# Load the target table
tgt_tbl = DeltaTable.forName(spark, target_table)

# Merge source into target
tgt_tbl.alias("tgt").merge(
    src_df.alias("src"),
    "tgt.CustomerID = src.Customer_ID"
).whenMatchedUpdate(
     condition="""
        tgt.CustomerName <> src.Customer_Name OR
        tgt.Segment <> src.Segment OR
        tgt.Country <> src.Country OR
        tgt.City <> src.City OR
        tgt.State <> src.State OR
        tgt.PostalCode <> src.Postal_Code OR
        tgt.Region <> src.Region
    """,
    set={
    "CustomerName": "src.Customer_Name",
    "Segment": "src.Segment",
    "Country": "src.Country",
    "City": "src.City",
    "State": "src.State",
    "PostalCode": "src.Postal_Code",
    "Region": "src.Region",
    "updatedDate": "current_timestamp()"
}).whenNotMatchedInsert(values={
    "CustomerID": "src.Customer_ID",
    "CustomerName": "src.Customer_Name",
    "Segment": "src.Segment",
    "Country": "src.Country",
    "City": "src.City",
    "State": "src.State",
    "PostalCode": "src.Postal_Code",
    "Region": "src.Region",
    "createdDate": "current_timestamp()",
    "updatedDate": "current_timestamp()"
}).execute()


In [0]:
%sql
Select * from  dim.Customer



CustomerID,CustomerName,Segment,Country,City,State,PostalCode,Region,createdDate,updatedDate
PK-19075,Pete Kriz,Consumer,United States,Madison,Wisconsin,53711,Central,2025-02-01T14:33:05.051Z,2025-02-01T14:33:05.051Z
MA-17560,Matt Abelman,Home Office,United States,Houston,Texas,77095,Central,2025-02-01T14:33:05.051Z,2025-02-01T14:33:05.051Z
ES-14080,Erin Smith,Corporate,United States,Melbourne,Florida,32935,South,2025-02-01T14:33:05.051Z,2025-02-01T14:33:05.051Z
PO-18865,Patrick O'Donnell,Consumer,United States,Westland,Michigan,48185,Central,2025-02-01T14:33:05.051Z,2025-02-01T14:33:05.051Z
EB-13870,Emily Burns,Consumer,United States,Orem,Utah,84057,West,2025-02-01T14:33:05.051Z,2025-02-01T14:33:05.051Z
SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,2025-02-01T14:33:05.051Z,2025-02-01T14:33:05.051Z
EH-13945,Eric Hoffmann,Consumer,United States,Los Angeles,California,90049,West,2025-02-01T14:33:05.051Z,2025-02-01T14:33:05.051Z
DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,2025-02-01T14:33:05.051Z,2025-02-01T14:33:05.051Z
IM-15070,Irene Maddox,Consumer,United States,Seattle,Washington,98103,West,2025-02-01T14:33:05.051Z,2025-02-01T14:33:05.051Z
GH-14485,Gene Hale,Corporate,United States,Richardson,Texas,75080,Central,2025-02-01T14:33:05.051Z,2025-02-01T14:33:05.051Z
